# Project Title
### Data Engineering Capstone Project

#### Project Summary
The main goal of this project is to build a data warehouse by integrating data from several sources for data analysis.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# import required libraries
import pandas as pd
import pyspark

### Step 1: Scope the Project and Gather Data

#### Scope 
Design and build a single source-of-truth data warehouse for data analysis purpose. The data warehouse will consist of fact and dimensional tables in which data are ingested and transformed from I94 immigration data, world temperature data, US cities demographic and airport code table 

#### Describe and Gather Data 

* [I94 Immigration Data](https://travel.trade.gov/research/reports/i94/historical/2016.html): This data comes from the US National Tourism and Trade Office in SAS format, which contains statistical information of international visitor arrival  by select countries and regions. 
* [World Temperature Data](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data): This dataset came from Kaggle in CSV format,storing monthly average temperature data at different countries worldwide.
* [U.S. City Demographic Data](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/): This data comes from OpenSoft in CSV format containing information about all US cities demographics.
* [Airport Code Table](https://datahub.io/core/airport-codes#data): This is a simple table of airport codes and corresponding cities.

###### I94 Immigration Data

In [2]:
# read i94 immigration data
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
immigration_df = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")
pd.options.display.max_columns = None
immigration_df.head()

# immigration_df = pd.read_csv("immigration_data_sample.csv")
# immigration_df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,37.0,2.0,1.0,NaN,NaN,NaN,T,NaN,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,25.0,3.0,1.0,20130811,SEO,NaN,G,NaN,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,55.0,2.0,1.0,20160401,NaN,NaN,T,O,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,28.0,2.0,1.0,20160401,NaN,NaN,O,O,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,4.0,2.0,1.0,20160401,NaN,NaN,O,O,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


###### World Temperature Data

In [3]:
# read temperature data
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
temperature_df = pd.read_csv(fname)
temperature_df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


###### US Cities Demographics Data

In [4]:
# read us cities demographics data
demographics_df = pd.read_csv("us-cities-demographics.csv", sep=";")
demographics_df.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


###### Airport Codes

In [5]:
# read airport codes data
airport_codes_df = pd.read_csv("airport-codes_csv.csv")
airport_codes_df.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


###### Read I94 Immigration Data by Pyspark

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [7]:
# write immigration to parquet files
# df_spark.write.parquet("sas_data/immigration")
immigration_df_spark=spark.read.parquet("sas_data/immigration")
immigration_df_spark.show(5)

+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|    cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|5748517.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     CA|20582.0|  40.0|    1.0|  1.0|20160430|     SYD| null|      G|      O|   null|      M| 1976.0|10292016|     F|  null|     QF|9.495387003E10|00011|      B1|
|5748518.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     NV|20591.0|  32.0|    1.0|  

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

###### Explore I94 Immigration Data

In [8]:
# view random 10 rows in i94 immigration dataframe
immigration_df.sample(5)

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
643443,1341557.0,2016.0,4.0,692.0,692.0,FMY,20551.0,1.0,NY,20565.0,41.0,2.0,1.0,20160407,GYQ,NaN,G,O,NaN,M,1975.0,10062016,M,NaN,AV,9.292447e+10,00020,B2
1589061,3239706.0,2016.0,4.0,582.0,582.0,PHO,20561.0,1.0,CA,NaN,50.0,2.0,1.0,20160417,GDL,NaN,G,NaN,NaN,NaN,1966.0,10162016,F,NaN,AA,9.378957e+10,00435,B2
1358237,2735580.0,2016.0,4.0,245.0,245.0,CHI,20559.0,1.0,CA,20577.0,27.0,2.0,1.0,20160415,GUZ,NaN,G,O,NaN,M,1989.0,10142016,F,NaN,CA,9.365094e+10,00983,B2
2592339,5244717.0,2016.0,4.0,148.0,112.0,ATL,20572.0,1.0,FL,20585.0,38.0,2.0,1.0,20160428,NaN,NaN,G,O,NaN,M,1978.0,07262016,M,NaN,AF,5.939656e+10,00682,WT
2492686,5051209.0,2016.0,4.0,148.0,112.0,MIA,20571.0,1.0,FL,20575.0,41.0,2.0,1.0,20160427,NaN,NaN,G,R,NaN,M,1975.0,07252016,F,NaN,AB,5.933744e+10,07000,WT


In [9]:
# number of rows and columns in i94 immigration dataframe
immigration_df.shape

(3096313, 28)

In [10]:
# statistical information of numeric columns in i94 immigration dataframe
immigration_df.describe()

,cicid,i94yr,i94mon,i94cit,i94res,arrdate,i94mode,depdate,i94bir,i94visa,count,biryear,admnum
count,3.096313e+06,3096313.0,3096313.0,3.096313e+06,3.096313e+06,3.096313e+06,3.096074e+06,2.953856e+06,3.095511e+06,3.096313e+06,3096313.0,3.095511e+06,3.096313e+06
mean,3.078652e+06,2016.0,4.0,3.049069e+02,3.032838e+02,2.055985e+04,1.073690e+00,2.057395e+04,4.176761e+01,1.845393e+00,1.0,1.974232e+03,7.082885e+10
std,1.763278e+06,0.0,0.0,2.100269e+02,2.085832e+02,8.777339e+00,5.158963e-01,2.935697e+01,1.742026e+01,3.983910e-01,0.0,1.742026e+01,2.215442e+10
min,6.000000e+00,2016.0,4.0,1.010000e+02,1.010000e+02,2.054500e+04,1.000000e+00,1.517600e+04,-3.000000e+00,1.000000e+00,1.0,1.902000e+03,0.000000e+00
25%,1.577790e+06,2016.0,4.0,1.350000e+02,1.310000e+02,2.055200e+04,1.000000e+00,2.056100e+04,3.000000e+01,2.000000e+00,1.0,1.962000e+03,5.603523e+10
50%,3.103507e+06,2016.0,4.0,2.130000e+02,2.130000e+02,2.056000e+04,1.000000e+00,2.057000e+04,4.100000e+01,2.000000e+00,1.0,1.975000e+03,5.936094e+10
75%,4.654341e+06,2016.0,4.0,5.120000e+02,5.040000e+02,2.056700e+04,1.000000e+00,2.057900e+04,5.400000e+01,2.000000e+00,1.0,1.986000e+03,9.350987e+10
max,6.102785e+06,2016.0,4.0,9.990000e+02,7.600000e+02,2.057400e+04,9.000000e+00,4.542700e+04,1.140000e+02,3.000000e+00,1.0,2.019000e+03,9.991557e+10


###### World Temperature Data

In [11]:
# view random 10 rows in temperature dataframe
temperature_df.sample(5)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
8234502,1943-10-01,10.594,0.331,Witten,Germany,52.24N,7.88E
7179967,1957-08-01,15.537,0.667,Stockholm,Sweden,58.66N,18.46E
8169721,1855-11-01,1.500,0.983,Waterloo,Canada,44.20N,80.50W
7674140,1982-03-01,-5.905,0.275,Tolyatti,Russia,53.84N,50.45E
1762875,1888-05-01,13.946,1.061,Coquimbo,Chile,29.74S,72.00W


In [12]:
# select only US data
temperature_df[temperature_df['Country']=='United States'].head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
47555,1820-01-01,2.101,3.217,Abilene,United States,32.95N,100.53W
47556,1820-02-01,6.926,2.853,Abilene,United States,32.95N,100.53W
47557,1820-03-01,10.767,2.395,Abilene,United States,32.95N,100.53W
47558,1820-04-01,17.989,2.202,Abilene,United States,32.95N,100.53W
47559,1820-05-01,21.809,2.036,Abilene,United States,32.95N,100.53W


In [13]:
# number of rows and columns in temperature dataframe
temperature_df.shape

(8599212, 7)

In [14]:
# statistical information of numeric columns in temperature dataframe
temperature_df.describe()

,AverageTemperature,AverageTemperatureUncertainty
count,8.235082e+06,8.235082e+06
mean,1.672743e+01,1.028575e+00
std,1.035344e+01,1.129733e+00
min,-4.270400e+01,3.400000e-02
25%,1.029900e+01,3.370000e-01
50%,1.883100e+01,5.910000e-01
75%,2.521000e+01,1.349000e+00
max,3.965100e+01,1.539600e+01


###### US Cities Demographics Data

In [15]:
# view random 10 rows in the US cities demographics dataframe
demographics_df.sample(5)

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
772,Youngs,Ohio,37.5,30758.0,33851.0,64609,3620.0,1058.0,2.22,OH,American Indian and Alaska Native,875
1765,West Jordan,Utah,30.8,57898.0,54044.0,111942,3577.0,11492.0,3.46,UT,Black or African-American,3179
425,Orange,California,35.0,67337.0,73658.0,140995,3993.0,34550.0,3.14,CA,Hispanic or Latino,46255
403,Orlando,Florida,33.1,130940.0,139977.0,270917,12782.0,50558.0,2.42,FL,Hispanic or Latino,89306


In [16]:
# number of rows and columns in the US cities demographics dataframe
demographics_df.shape

(2891, 12)

In [17]:
# statistical information of numeric columns in the US cities demographics dataframe
demographics_df.describe()

,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,Count
count,2891.000000,2.888000e+03,2.888000e+03,2.891000e+03,2878.000000,2.878000e+03,2875.000000,2.891000e+03
mean,35.494881,9.732843e+04,1.017696e+05,1.989668e+05,9367.832523,4.065360e+04,2.742543,4.896377e+04
std,4.401617,2.162999e+05,2.315646e+05,4.475559e+05,13211.219924,1.557491e+05,0.433291,1.443856e+05
min,22.900000,2.928100e+04,2.734800e+04,6.321500e+04,416.000000,8.610000e+02,2.000000,9.800000e+01
25%,32.800000,3.928900e+04,4.122700e+04,8.042900e+04,3739.000000,9.224000e+03,2.430000,3.435000e+03
50%,35.300000,5.234100e+04,5.380900e+04,1.067820e+05,5397.000000,1.882200e+04,2.650000,1.378000e+04
75%,38.000000,8.664175e+04,8.960400e+04,1.752320e+05,9368.000000,3.397175e+04,2.950000,5.444700e+04
max,70.500000,4.081698e+06,4.468707e+06,8.550405e+06,156961.000000,3.212500e+06,4.980000,3.835726e+06


###### Airport Codes

In [18]:
# view random 10 rows in airport codes dataframe
airport_codes_df.sample(5)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
33871,MS39,small_airport,Smith County Airport,332.0,NaN,US,US-MS,Taylorsville,MS39,NaN,MS39,"-89.4583969116211, 31.820999145507812"
10573,AK99,small_airport,Grand Home Airport,85.0,NaN,US,US-AK,Wasilla,AK99,NaN,AK99,"-149.761111, 61.446667"
28670,KPWD,small_airport,Sher-Wood Airport,2250.0,NaN,US,US-MT,Plentywood,KPWD,PWD,PWD,"-104.53399658203, 48.790298461914"
8105,79PN,heliport,Geisinger Rooftop Heliport,661.0,NaN,US,US-PA,Danville,79PN,NaN,79PN,"-76.6041030883789, 40.972900390625"
37762,OOKB,medium_airport,Khasab Air Base,100.0,AS,OM,OM-MU,Khasab,OOKB,KHS,NaN,"56.2406005859375, 26.17099952697754"


In [19]:
# number of rows and columns in airport codes dataframe
airport_codes_df.shape

(55075, 12)

In [20]:
# statistical information of numeric columns in airport codes dataframe
airport_codes_df.describe()

,elevation_ft
count,48069.000000
mean,1240.789677
std,1602.363459
min,-1266.000000
25%,205.000000
50%,718.000000
75%,1497.000000
max,22000.000000


#### Cleaning Steps
1. Drop unwanted, add new extract and rename columns
2. Transform arrival and departure dates in immigration data from SAS time format to Pandas datetime format
3. Parse I94_SAS_Labels_Descriptions.SAS file to obtain country_code, state_code and city_code

###### Explore I94 Immigration Data

In [21]:
# drop unwanted and rename columns

immigration_df = immigration_df[['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port', 'arrdate', 'i94mode', 'i94addr', \
                                 'depdate', 'i94visa', 'visapost', 'biryear', 'gender', 'airline', 'admnum', 'fltno', 'visatype']]

immigration_df.columns = ['cic_id', 'year', 'month', 'citizens_country_code', 'residential_country_code', 'port_code', 'arrive_date', 'mode', 'address', \
                          'departure_date', 'visa', 'visa_post', 'birth_year', 'gender', 'airline', 'admin_num', 'flight_number', 'visa_type']
immigration_df.dropna(inplace=True)
immigration_df.head()

,cic_id,year,month,citizens_country_code,residential_country_code,port_code,arrive_date,mode,address,departure_date,visa,visa_post,birth_year,gender,airline,admin_num,flight_number,visa_type
12,27.0,2016.0,4.0,101.0,101.0,BOS,20545.0,1.0,MA,20549.0,1.0,TIA,1958.0,M,LH,9.247876e+10,00422,B1
13,28.0,2016.0,4.0,101.0,101.0,ATL,20545.0,1.0,MA,20549.0,1.0,TIA,1960.0,F,LH,9.247890e+10,00422,B1
14,29.0,2016.0,4.0,101.0,101.0,ATL,20545.0,1.0,MA,20561.0,2.0,TIA,1954.0,M,AZ,9.250378e+10,00614,B2
15,30.0,2016.0,4.0,101.0,101.0,ATL,20545.0,1.0,NJ,20578.0,2.0,TIA,1967.0,M,OS,9.247021e+10,00089,B2
16,31.0,2016.0,4.0,101.0,101.0,ATL,20545.0,1.0,NY,20611.0,2.0,TIA,1973.0,M,OS,9.247129e+10,00089,B2


###### World Temperature Data

In [22]:
# select only US data
temperature_us_df = temperature_df[temperature_df['Country']=='United States'].reset_index(drop=True)
temperature_us_df.dropna(inplace=True)

# add year and month columns
temperature_us_df['dt'] = pd.to_datetime(temperature_us_df['dt'])
temperature_us_df['year'] = temperature_us_df['dt'].dt.year
temperature_us_df['month'] = temperature_us_df['dt'].dt.month

# rename columns
temperature_us_df.columns = ['dt', 'avg_temperature', 'avg_temperature_uncertainty', 'city', 'country', 'latitude', 'longitude', 'year', 'month']
temperature_us_df.head()

,dt,avg_temperature,avg_temperature_uncertainty,city,country,latitude,longitude,year,month
0,1820-01-01,2.101,3.217,Abilene,United States,32.95N,100.53W,1820,1
1,1820-02-01,6.926,2.853,Abilene,United States,32.95N,100.53W,1820,2
2,1820-03-01,10.767,2.395,Abilene,United States,32.95N,100.53W,1820,3
3,1820-04-01,17.989,2.202,Abilene,United States,32.95N,100.53W,1820,4
4,1820-05-01,21.809,2.036,Abilene,United States,32.95N,100.53W,1820,5


###### US Cities Demographics Data

In [23]:
# rename columns
demographics_df.dropna(inplace=True)
demographics_df.columns = ['city', 'state','median_age', 'male_population', 'female_population', 'total_population', \
                           'veterans_number', 'foreign_born', 'avg_household_size', 'state_code', 'race', 'count']
demographics_df.head()

,city,state,median_age,male_population,female_population,total_population,veterans_number,foreign_born,avg_household_size,state_code,race,count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


###### Airport Codes

In [24]:
# rename columns
airport_codes_df.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


##### Transform SAS date format to standard date format

In [25]:
# define transform function
def sas_date_transform(date):
    return pd.to_timedelta(date, unit='D') + pd.Timestamp('1960-1-1')

In [26]:
immigration_df['arrive_date'] = sas_date_transform(immigration_df['arrive_date']) 
immigration_df['departure_date'] = sas_date_transform(immigration_df['departure_date']) 
immigration_df.head()

,cic_id,year,month,citizens_country_code,residential_country_code,port_code,arrive_date,mode,address,departure_date,visa,visa_post,birth_year,gender,airline,admin_num,flight_number,visa_type
12,27.0,2016.0,4.0,101.0,101.0,BOS,2016-04-01,1.0,MA,2016-04-05,1.0,TIA,1958.0,M,LH,9.247876e+10,00422,B1
13,28.0,2016.0,4.0,101.0,101.0,ATL,2016-04-01,1.0,MA,2016-04-05,1.0,TIA,1960.0,F,LH,9.247890e+10,00422,B1
14,29.0,2016.0,4.0,101.0,101.0,ATL,2016-04-01,1.0,MA,2016-04-17,2.0,TIA,1954.0,M,AZ,9.250378e+10,00614,B2
15,30.0,2016.0,4.0,101.0,101.0,ATL,2016-04-01,1.0,NJ,2016-05-04,2.0,TIA,1967.0,M,OS,9.247021e+10,00089,B2
16,31.0,2016.0,4.0,101.0,101.0,ATL,2016-04-01,1.0,NY,2016-06-06,2.0,TIA,1973.0,M,OS,9.247129e+10,00089,B2


##### Parse SAS Labels Descriptions File to Get country_code, city_code and state_code for Auxiliary Dimension Tables

In [27]:
# Read SAS text file
with open("./I94_SAS_Labels_Descriptions.SAS") as f:
    content = f.readlines()

In [28]:
# Get country_code
country_code = []
for line in content[10:298]:
    line = line.split("=")
    code = line[0].strip()
    country = line[1].strip().strip("' ;")
    country_code.append([code, country])
    
# Create country_code dataframe
country_code_df = pd.DataFrame(country_code, columns=["country_code", "country"])
country_code_df.head()
# country_code_df.tail()

,country_code,country
0,236,AFGHANISTAN
1,101,ALBANIA
2,316,ALGERIA
3,102,ANDORRA
4,324,ANGOLA


In [29]:
# Get state code
state_code = []
for line in content[981:1036]:
    line = line.split("=")
    code = line[0].strip().strip("','")
    state = line[1].strip().strip("' ;")
    state_code.append([code, state])

# Create state_code dataframe
state_code_df = pd.DataFrame(state_code, columns=['state_code', 'state'])
state_code_df.head()
# state_code_df.tail()

,state_code,state
0,AL,ALABAMA
1,AK,ALASKA
2,AZ,ARIZONA
3,AR,ARKANSAS
4,CA,CALIFORNIA


In [30]:
# Get port information
port = []
for line in content[302:962]:
    line = line.split("=")
    port_code = line[0].strip("\t").strip().strip("''")
    city = line[1].split(",")[0].strip().strip("''")
    state_code = line[1].split(",")[-1].strip().strip("''")
    port.append([port_code, city, state_code])   

# Create port_code dataframe
port_infor_df = pd.DataFrame(port, columns=["port_code", "city", "state_code"])
port_infor_df.head()
# port_infor_df.tail()

,port_code,city,state_code
0,ALC,ALCAN,AK
1,ANC,ANCHORAGE,AK
2,BAR,BAKER AAF - BAKER ISLAND,AK
3,DAC,DALTONS CACHE,AK
4,PIZ,DEW STATION PT LAY DEW,AK


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Since the purpose of this data warehouse is for data analysis usage, we will model these data sets with star schema data modeling. The fact, dimensional and auxilary tables has been designed as the following diagram.

<img src="./capstoneprojectdb_erd.png" width="88%"/>

#### 3.2 Mapping Out Data Pipelines
1. Read data from various sources
2. Follow by Step 2 – Cleaning step to clean up data sets
3. Parsing label description file to get data for auxiliary tables
4. Create data warehouse and tables 
5. Insert processed data into fact, dimention and auxilary tables in data warehouse

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

To start the elt pipeline:
1. Open terminal
2. Run create_tables.py file
3. Then run etl.py file

#### 4.2 Data Quality Checks

No empty table after runing ETL

In [31]:
# Connect to the data warehouse
import psycopg2
conn = psycopg2.connect("host=127.0.0.1 dbname=capstoneprojectdb user=student password=student")
cur = conn.cursor()

# check fact_immigration table
cur.execute("SELECT COUNT(*) FROM fact_immigration")
conn.commit()
if cur.rowcount < 1:
    print("No data found in fact_immigration table")
    
# check dim_temperature table
cur.execute("SELECT COUNT(*) FROM dim_temperature")
conn.commit()
if cur.rowcount < 1:
    print("No data found in dim_temperature table")

# check dim_demographics table
cur.execute("SELECT COUNT(*) FROM dim_demographics")
conn.commit()
if cur.rowcount < 1:
    print("No data found in dim_demographics table")
    
# check dim_demographics table
cur.execute("SELECT COUNT(*) FROM dim_airport")
conn.commit()
if cur.rowcount < 1:
    print("No data found in dim_airport table")
    
# check country_code table    
cur.execute("SELECT COUNT(*) FROM country_code")
conn.commit()
if cur.rowcount < 1:
    print("No data found in country_code table")
    
# check state_code table    
cur.execute("SELECT COUNT(*) FROM state_code")
conn.commit()
if cur.rowcount < 1:
    print("No data found in state_code table")

# check port_infor table
cur.execute("SELECT COUNT(*) FROM port_infor")
conn.commit()
if cur.rowcount < 1:
    print("No data found in port_infor table")

# close the connection
conn.close()

#### 4.3 Data dictionary 

<img src="./data_dictionaries.png" width="88%"/>

#### Step 5: Complete Project Write Up
* The choice of tools and technologies for the project.
1. Postgress for data storage
2. Pandas for sample data set exploratory data analysis

* Propose how often the data should be updated and why.
1. Tables created from immigration and temperature data set should be updated monthly since the raw data set is built up monthly.
2. Tables created from demography data set could be updated annually since demography data collection takes time and high frequent demography might take high cost but generate wrong conclusion.

* Write a description of how you would approach the problem differently under the following scenarios:
1. The data was increased by 100x.
If python on local machine can not process 100x data set, we could consider to use a distributed cluster for processing large data sets on cloud, like AWS EMR
2. The data populates a dashboard that must be updated on a daily basis by 7am every day.
Apache Airflow could be used for building up a ETL data pipeline to regularly update the date and populate a report. 
3. The database needed to be accessed by 100+ people.
If this database will be accessed by 100+ people, we can move this database to Redshift with confidence to handle this request.